In [3]:
from bs4 import BeautifulSoup
import requests
import re
from retry import retry 
import pandas as pd
from lxml import etree
from selenium import webdriver
from selenium.webdriver.common.by import By
import re


# 必要な関数類
@retry(tries=3, delay=10, backoff=2)
def get_html(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    return soup

def keyword_to_url(keywords):
    first = "https://job.mynavi.jp/24/pc/corpinfo/searchCorpListByGenCond/index?actionMode=searchFw&cond=&"
    last = f"srchWord={keywords}&q={keywords}&SC=corp&srchWordTgt=1"
    return first+last

def make_id_dict(elem_list):
    result = {}
    for item in elem_list:
        _id = re.search("(.*)(_label)", item.get("id"))[1]
        result[item.text] = _id
    return result


In [24]:
# driverを使って自動でchromeを操作する。
# コードの使用にはドライバーのインストールが必要 https://chromedriver.chromium.org/downloads
# 以下のコード第一引数はドライバーの絶対パス
driver = webdriver.Chrome('/Users/ricky/dev/scraping/drivers/chromedriver')

/var/folders/9z/_tv0kl191wvbx_078d5dg6fc0000gn/T/ipykernel_58550/3505965420.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/ricky/dev/scraping/drivers/chromedriver')


In [195]:
# 条件検索は未実装なので、とりあえずキーワードだけ
# キーワードはスペース区切りで "製造 メーカー" のように入力する
keyword_list = ["なのちゃん", "製造 メーカー", "製造", "非鉄金属", "メーカー", "商社", "精密機器", "東京都"]
data = []

for keyword in keyword_list:
    print("=====================================================================")
    print(f"Keyword : {keyword}")
    url = keyword_to_url(keyword)
    driver.get(url)
    result_num = int(driver.find_element(By.ID, "searchResultkensuu").text)
    print(f"Search results number : {result_num}")
    next_page = True
    found = False
    page = 0
    while next_page and not found:
        print(f"Searching... page {page}")
        html = BeautifulSoup(driver.page_source, "html.parser")
        a_tags = html.find_all("a", {"id":re.compile("corpNameLink\[\d+\]")})
        for num, a in enumerate(a_tags):
            if a.text == "タキゲン製造(株)":
                order = int(page * 100 + num + 1)
                row = [keyword, result_num, True, order, round(order/result_num, 4)*100]
                print(f"New data gained : {row}")
                data.append(row)
                found = True
                break
        try:
            next_button = driver.find_element(By.ID, "upperNextPage")
            next_button.click()
            page += 1
        except:
            next_page = False
    if not found:
        row = [keyword, result_num, False, None, None]
        print(f"New data gained : {row}")
        data.append(row)

columns=["検索語", "検索結果数", "結果の有無", "順位", "検索結果の上位%"]
df = pd.DataFrame(data, columns=columns)


Keyword : なのちゃん
Search results number : 0
Searching... page 0
New data gained : ['なのちゃん', 0, False, None, None]
Keyword : 製造 メーカー
Search results number : 1246
Searching... page 0
Searching... page 1
Searching... page 2
Searching... page 3
Searching... page 4
Searching... page 5
Searching... page 6
Searching... page 7
Searching... page 8
Searching... page 9
Searching... page 10
Searching... page 11
Searching... page 12
New data gained : ['製造 メーカー', 1246, False, None, None]
Keyword : 製造
Search results number : 8794
Searching... page 0
New data gained : ['製造', 8794, True, 15, 0.0017]
Keyword : 非鉄金属
Search results number : 519
Searching... page 0
Searching... page 1
Searching... page 2
New data gained : ['非鉄金属', 519, True, 252, 0.4855]
Keyword : メーカー
Search results number : 4998
Searching... page 0
Searching... page 1
Searching... page 2
Searching... page 3
Searching... page 4
Searching... page 5
Searching... page 6
Searching... page 7
Searching... page 8
Searching... page 9
Searching... p

In [194]:
df

,Search word,Num of results,Found,Order
0,なのちゃん,0,False,NaN
1,製造 メーカー,1118,False,NaN
2,製造,8536,True,22.0
3,非鉄金属,502,True,341.0
4,メーカー,4696,True,2043.0
5,商社,7013,True,4563.0
6,精密機器,1223,True,853.0
7,東京都,4148,True,3090.0


In [193]:
# 条件検索用(未実装)、APIをできるだけ使いやすくして実装したいなと考え中。1時間もあればできそう
# 条件検索用の情報１
driver.get("https://job.mynavi.jp/24/pc/corpinfo/displayCorpSearch/index")
html = BeautifulSoup(driver.page_source, "html.parser")
ind_category_inputs = html.find_all("a", {"id":re.compile(r"industryCheckLink\d+")})
ind_detail_inputs = html.find_all("a", {"id":re.compile(r"industryCtgDetailedCheckArray\d+")})
area_inputs = html.find_all("a", {"id": re.compile(r"ifRegional\d*")})
occu_inputs = html.find_all("a", {"id": re.compile("occGroup\d+")})
corp_welfare_inputs = html.find_all("label", {"id":re.compile(r"corpWelfare\d+")})
emp_inputs = html.find_all("label", {"id":re.compile(r"empInfo\d+")}) 

In [192]:
# 条件検索用の情報２

conditions = {
    "職種(カテゴリ)": make_id_dict(ind_category_inputs), 
    "職種(詳細)" : make_id_dict(ind_detail_inputs), 
    "地域": make_id_dict(area_inputs), 
    "職種": make_id_dict(occu_inputs), 
    "福利厚生": make_id_dict(corp_welfare_inputs),
    "従業員数": make_id_dict(emp_inputs), 
}

conditions

{'職種(詳細)': {'農林・水産': 'industryCtgDetailedCheckArray100',
  '食品': 'industryCtgDetailedCheckArray110',
  '建設': 'industryCtgDetailedCheckArray120',
  '設備工事': 'industryCtgDetailedCheckArray121',
  '建築設計': 'industryCtgDetailedCheckArray122',
  '建設コンサルタント': 'industryCtgDetailedCheckArray123',
  '住宅': 'industryCtgDetailedCheckArray130',
  '住宅（リフォーム）': 'industryCtgDetailedCheckArray131',
  '建材・エクステリア': 'industryCtgDetailedCheckArray132',
  'インテリア・住宅関連': 'industryCtgDetailedCheckArray133',
  '空間デザイン・ディスプレイ': 'industryCtgDetailedCheckArray134',
  'アパレル（メーカー）': 'industryCtgDetailedCheckArray140',
  '繊維': 'industryCtgDetailedCheckArray150',
  '紙・パルプ': 'industryCtgDetailedCheckArray151',
  '化学': 'industryCtgDetailedCheckArray160',
  '石油': 'industryCtgDetailedCheckArray161',
  '薬品': 'industryCtgDetailedCheckArray170',
  '化粧品': 'industryCtgDetailedCheckArray171',
  'タイヤ・ゴム製品': 'industryCtgDetailedCheckArray180',
  'セメント': 'industryCtgDetailedCheckArray181',
  'ガラス・セラミックス': 'industryCtgDetailedCheckAr